# Tensorflow Model - To Predict Music Genre

Inputs:
- acousticness
- danceability	
- energy
- instrumentalness
- key
- liveness
- loudness
- mode
- speechiness
- tempo
- valence

Outputs: Music Genre


# Jupyter Notebook Shortcuts on VSCode
### Command Mode
Edit mode - Enter

Command mode - ESC

### Cell Type
_In command mode_

Markdown - M

Code - Y

### Add cell
_In command mode_

Add above - A

Add below - B

### Run a single cell
curr cell - Ctrl+Enter

curr cell, add next cell, focus to new cell - Shift+Enter

curr cell, add next cell, focus remain curr cell - Alt+Enter



In [59]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [60]:
# import data
musicData = pd.read_csv("SpotifyFeatures.csv")
musicData = musicData.drop(['artist_name', 'track_name', 'popularity', 'track_id', 'key', 'mode', 'time_signature'], axis=1)

In [61]:
musicData.head()

,genre,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,A Capella,0.780,0.616,223813,0.298,0.000000,0.0973,-7.712,0.0274,82.471,0.712
1,A Capella,0.973,0.484,154200,0.222,0.000306,0.1040,-14.631,0.0440,127.689,0.409
2,A Capella,0.802,0.717,137800,0.518,0.000000,0.0940,-7.754,0.0883,129.878,0.672
3,A Capella,0.302,0.691,183920,0.490,0.000006,0.2290,-11.222,0.0743,107.066,0.664
4,A Capella,0.599,0.389,181267,0.153,0.000000,0.0902,-12.023,0.0430,120.626,0.236


In [62]:
musicData.dtypes

genre                object
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
tempo               float64
valence             float64
dtype: object

In [63]:
# convert genre which is an object in the dataframe to a discrete value
musicData['genre'] = pd.Categorical(musicData['genre'])
musicData['genre'] = musicData.genre.cat.codes
musicData['genre']

0          0
1          0
2          0
3          0
4          0
          ..
232720    25
232721    25
232722    25
232723    25
232724    25
Name: genre, Length: 232725, dtype: int8

In [64]:
musicData.shape
# 116363+116362

(232725, 11)

In [65]:
# Split data into training data and test data
musicTrainingData = musicData.iloc[::2, :]
musicTrainingData

,genre,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0,0.7800,0.616,223813,0.2980,0.000,0.0973,-7.712,0.0274,82.471,0.7120
2,0,0.8020,0.717,137800,0.5180,0.000,0.0940,-7.754,0.0883,129.878,0.6720
4,0,0.5990,0.389,181267,0.1530,0.000,0.0902,-12.023,0.0430,120.626,0.2360
6,0,0.6680,0.747,135107,0.5050,0.000,0.1010,-9.263,0.0746,122.177,0.7160
8,0,0.8350,0.583,175000,0.3180,0.000,0.1180,-8.252,0.0311,75.911,0.5100
...,...,...,...,...,...,...,...,...,...,...,...
232716,25,0.8120,0.106,337787,0.1820,0.949,0.1150,-20.766,0.0348,168.517,0.0372
232718,25,0.8120,0.252,358573,0.2580,0.889,0.1070,-15.485,0.0336,81.059,0.0481
232720,25,0.9790,0.347,212373,0.0239,0.925,0.1190,-33.799,0.0286,94.420,0.1540
232722,25,0.0684,0.640,184960,0.5700,0.000,0.0709,-6.554,0.1590,180.146,0.3160


In [66]:
# Split data into training data and test data
musicTestData = musicData.iloc[1::2, :]
musicTestData

,genre,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
1,0,0.97300,0.484,154200,0.2220,0.000306,0.1040,-14.631,0.0440,127.689,0.4090
3,0,0.30200,0.691,183920,0.4900,0.000006,0.2290,-11.222,0.0743,107.066,0.6640
5,0,0.79300,0.349,228147,0.0913,0.000002,0.1410,-15.003,0.0396,120.405,0.1500
7,0,0.65600,0.705,159120,0.5700,0.000000,0.0897,-7.124,0.0983,81.977,0.6520
9,0,0.36000,0.654,150027,0.4270,0.000000,0.1500,-7.963,0.0379,125.390,0.8110
...,...,...,...,...,...,...,...,...,...,...,...
232715,25,0.97800,0.133,162333,0.1690,0.886000,0.0814,-15.312,0.0415,99.596,0.0313
232717,25,0.00202,0.413,250347,0.8360,0.000000,0.0842,-4.867,0.0433,143.962,0.3030
232719,25,0.12100,0.634,220787,0.8660,0.000001,0.0641,-4.987,0.1470,106.794,0.6530
232721,25,0.99500,0.373,101533,0.0270,0.871000,0.0854,-17.503,0.0390,110.866,0.0905


In [67]:
# Batch and Shuffle Dataset
# train_ds = tf.data.Dataset.from_tensor_slices()

In [68]:
# Seperate feautures and Labels
musicFeatures = musicTrainingData.copy()
musicLabels = musicTrainingData.pop('genre')

musicFeatures = np.array(musicFeatures)
musicFeatures

array([[0.00000e+00, 7.80000e-01, 6.16000e-01, ..., 2.74000e-02,
        8.24710e+01, 7.12000e-01],
       [0.00000e+00, 8.02000e-01, 7.17000e-01, ..., 8.83000e-02,
        1.29878e+02, 6.72000e-01],
       [0.00000e+00, 5.99000e-01, 3.89000e-01, ..., 4.30000e-02,
        1.20626e+02, 2.36000e-01],
       ...,
       [2.50000e+01, 9.79000e-01, 3.47000e-01, ..., 2.86000e-02,
        9.44200e+01, 1.54000e-01],
       [2.50000e+01, 6.84000e-02, 6.40000e-01, ..., 1.59000e-01,
        1.80146e+02, 3.16000e-01],
       [2.50000e+01, 2.49000e-02, 8.28000e-01, ..., 8.79000e-02,
        1.21167e+02, 5.37000e-01]])

In [69]:
# train_loss = tf.keras.metrics.Mean(name='train_loss')
# train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# test_loss = tf.keras.metrics.Mean(name='test_loss')
# test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [74]:
# TODO: Might need to change the model for higher accuracy, lower loss

# Regression Model
# Try single input tensor with keras.Sequential
musicModel = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
])

musicModel.compile(
    loss = tf.losses.MeanSquaredError(),
    optimizer= tf.optimizers.Adam(),
    metrics = tf.metrics.CategoricalCrossentropy()

    # loss = tf.losses.SparseCategoricalCrossentropy, 
    # optimizer= tf.optimizers.RMSprop(),
    # metrics= tf.metrics.SparseCategoricalAccuracy()
)

In [75]:
# Train model
musicModel.fit(musicFeatures, musicLabels, epochs=10)

Epoch 1/10
3637/3637 [==============================] - 6s 2ms/step - loss: 1709170.3750 - categorical_crossentropy: 1.5216e-06
Epoch 2/10
3637/3637 [==============================] - 7s 2ms/step - loss: 253335.3281 - categorical_crossentropy: 1.5216e-06
Epoch 3/10
3637/3637 [==============================] - 7s 2ms/step - loss: 38210.5977 - categorical_crossentropy: 1.5216e-06
Epoch 4/10
3637/3637 [==============================] - 5s 1ms/step - loss: 84499.2656 - categorical_crossentropy: 1.5216e-06
Epoch 5/10
3637/3637 [==============================] - 5s 1ms/step - loss: 78242.1641 - categorical_crossentropy: 1.5216e-06
Epoch 6/10
3637/3637 [==============================] - 9s 2ms/step - loss: 190365.5938 - categorical_crossentropy: 1.5216e-06
Epoch 7/10
3637/3637 [==============================] - 6s 2ms/step - loss: 54403.6797 - categorical_crossentropy: 1.5216e-06
Epoch 8/10
3637/3637 [==============================] - 5s 1ms/step - loss: 126239.0469 - categorical_crossentropy

In [76]:
# Test model
musicFeatures = musicTestData.copy()
musicLabels = musicTestData.pop('genre')

musicModel.fit(musicFeatures, musicLabels, epochs=10)

Epoch 1/10
3637/3637 [==============================] - 10s 3ms/step - loss: 198553.2188 - categorical_crossentropy: 1.5216e-06
Epoch 2/10
3637/3637 [==============================] - 13s 4ms/step - loss: 26868.7168 - categorical_crossentropy: 1.5216e-06
Epoch 3/10
3637/3637 [==============================] - 9s 3ms/step - loss: 941514.1250 - categorical_crossentropy: 1.5216e-06
Epoch 4/10
3637/3637 [==============================] - 7s 2ms/step - loss: 3665.2515 - categorical_crossentropy: 1.5216e-06
Epoch 5/10
3637/3637 [==============================] - 10s 3ms/step - loss: 97610.5781 - categorical_crossentropy: 1.5216e-06
Epoch 6/10
3637/3637 [==============================] - 7s 2ms/step - loss: 110473.3828 - categorical_crossentropy: 1.5216e-06
Epoch 7/10
3637/3637 [==============================] - 8s 2ms/step - loss: 84753.6406 - categorical_crossentropy: 1.5216e-06
Epoch 8/10
3637/3637 [==============================] - 8s 2ms/step - loss: 529058.3125 - categorical_crossentrop